# Image Classification 


In [25]:
# Import numpy for array operations
import numpy as np

# To check Image Augmentation

In [19]:
# Import ImageDataGenerator for image preprocessing/augmentation
# This basically creates multiple copies of train images by jittering(adding noise). 
# This includes rotating, zooming in, flipping, shifting, etc.
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest') # 'nearest' is kind of algorithm to fill pixel values while transformation

In [20]:
import os
from keras.preprocessing.image import img_to_array, load_img

img = load_img('train/train/airplanes/image_0001.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (480, 640, 3)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 480, 640, 3)

# Create a directory named 'preview' in which we can save augmented images. 
os.system('mkdir preview')

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='airplanes', save_format='jpg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [21]:
img.size

(398, 164)

# CNN 

In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_12 (Activation)   (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)        36928     
__________

In [23]:
batch_size = 32

# This is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# This is the augmentation configuration we will use for testing:
# Only rescaling. Other transformations are not required for testing. Duh!
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'train/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'train/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 3724 images belonging to 10 classes.
Found 251 images belonging to 10 classes.


In [24]:
# fit_generator is similar to 'fit'. But instead of x_train and y_train, we pass 'train_generator' 
# which already has the samples and their corresponding target information.

# 'step' here is one mini-batch
# 'steps_per_epoch' is number of batches per epoch.
# Typically 'steps_per_epoch' should be total_samples divided by batch_size
model.fit_generator(
        train_generator,
        steps_per_epoch=120//batch_size, # '//' in python returns only the quotient
        epochs=50,
        validation_data=validation_generator,
        validation_steps=80//batch_size)
model.save_weights('model_img.h5')  # always save your weights after training or during training

Epoch 1/50
3/3 [==============================] - 30s 10s/step - loss: 3.4128 - acc: 0.1146 - val_loss: 2.3033 - val_acc: 0.1036
Epoch 2/50
3/3 [==============================] - 31s 10s/step - loss: 2.2818 - acc: 0.1250 - val_loss: 2.4676 - val_acc: 0.1315
Epoch 3/50
3/3 [==============================] - 34s 11s/step - loss: 2.1541 - acc: 0.2500 - val_loss: 2.3567 - val_acc: 0.2191
Epoch 4/50
3/3 [==============================] - 36s 12s/step - loss: 2.2061 - acc: 0.2083 - val_loss: 2.4593 - val_acc: 0.0956
Epoch 5/50
3/3 [==============================] - 32s 11s/step - loss: 2.1970 - acc: 0.2396 - val_loss: 2.2502 - val_acc: 0.1036
Epoch 6/50
3/3 [==============================] - 31s 10s/step - loss: 2.0516 - acc: 0.3229 - val_loss: 2.1992 - val_acc: 0.2151
Epoch 7/50
3/3 [==============================] - 30s 10s/step - loss: 1.9552 - acc: 0.3646 - val_loss: 2.1456 - val_acc: 0.1833
Epoch 8/50
3/3 [==============================] - 32s 11s/step - loss: 2.0919 - acc: 0.3542 - val

In [26]:
# print(train_generator.filenames)
print(train_generator.class_indices) # This returns class labels of directories

{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}


In [97]:
test_generator = test_datagen.flow_from_directory(
        'test/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical', shuffle = False)  # since we use crossentropy loss, we need binary labels

Found 32 images belonging to 1 classes.


In [98]:
test_prob = model.predict_generator(test_generator, steps=1) # this returns the probabilities
test_pred_classes = np.argmax(test_prob, axis=1) # convert probabilities to classes
print(test_pred_classes)

[2 2 2 7 4 4 4 4 4 4 4 4 2 4 4 4 4 0 8 4 4 5 4 4 4 4 4 4 4 4 4 4]


In [99]:
# Check the corresponding filenames of the predictions
print(test_generator.filenames[0])

test\image_0002.jpg


In [100]:
import pandas as pd 
test_predictions = pd.DataFrame(data=np.vstack((test_generator.filenames, test_pred_classes)).T, 
                                columns=['file', 'label'])

test_predictions.to_csv('test_submission.csv', header=True, sep=',')